In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import urllib.request

# -----------------------------------------------------------------------------
# 1. Load the pre-trained detection model from TensorFlow Hub
# -----------------------------------------------------------------------------
# We use the SSD MobileNet V2 model, which is a good trade-off of speed and accuracy.
print("Loading model…")
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
# The detector callable expects a batch of images as uint8 tensors in [0,255]

# -----------------------------------------------------------------------------
# 2. Load COCO class labels
# -----------------------------------------------------------------------------
# The model was trained on COCO; these are the 90 class names in order.
LABELS_URL = "https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt"
with urllib.request.urlopen(LABELS_URL) as f:
    class_names = f.read().decode("utf-8").splitlines()
# class_names[0] is "__background__"; detections start at index 1.

# -----------------------------------------------------------------------------
# 3. Utility: Run detector on a single frame
# -----------------------------------------------------------------------------
def detect_objects(frame):
    """
    Runs the detector on a frame.
    - frame: BGR image from OpenCV, uint8
    Returns a dict with:
      'detection_boxes' : [N,4] in normalized coords (ymin, xmin, ymax, xmax)
      'detection_classes' : [N] class indices (1‐90)
      'detection_scores' : [N] confidence scores
    """
    # Convert BGR→RGB, add batch dim, and build tensor
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)[tf.newaxis, ...]
    # Run inference
    results = detector(input_tensor)
    # Each entry is a [1,N,...] tensor; take the 0th batch element
    return {key: value[0].numpy() for key, value in results.items()}

# -----------------------------------------------------------------------------
# 4. Utility: Draw bounding boxes and labels on frame
# -----------------------------------------------------------------------------
def draw_boxes(frame, detections, thresh=0.5):
    """
    Draws boxes on the frame for all detections above thresh.
    Modifies frame in‐place.
    """
    h, w, _ = frame.shape
    boxes = detections['detection_boxes']
    classes = detections['detection_classes'].astype(int)
    scores = detections['detection_scores']

    for box, cls, score in zip(boxes, classes, scores):
        if score < thresh:
            continue
        ymin, xmin, ymax, xmax = box
        x1, y1, x2, y2 = int(xmin*w), int(ymin*h), int(xmax*w), int(ymax*h)

        # Draw rectangle
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Prepare label text
        label = f"{class_names[cls]}: {score:.2f}"
        # Draw label background
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        cv2.rectangle(frame, (x1, y1 - 20), (x1 + tw, y1), (0, 255, 0), -1)
        # Put label text
        cv2.putText(frame, label, (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

# -----------------------------------------------------------------------------
# 5. Main loop: Capture from camera and display detections
# -----------------------------------------------------------------------------
def main():
    # Open default camera (0); change index if you have multiple cameras
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("ERROR: Could not open camera.")
        return

    print("Starting camera. Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 1) Run detection
        detections = detect_objects(frame)

        # 2) Draw boxes above confidence threshold
        draw_boxes(frame, detections, thresh=0.5)

        # 3) Show the frame
        cv2.imshow("Object Detection", frame)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Clean up
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loading model…
Starting camera. Press 'q' to quit.


KeyboardInterrupt: 

: 